<a href="https://colab.research.google.com/github/yuji-matsunami/seq2seq_paractice/blob/main/seq2seq_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp /content/drive/MyDrive/nucc.zip .
!unzip nucc.zip

Archive:  nucc.zip
   creating: nucc/
  inflating: nucc/data001.txt        
  inflating: nucc/data002.txt        
  inflating: nucc/data003.txt        
  inflating: nucc/data004.txt        
  inflating: nucc/data005.txt        
  inflating: nucc/data006.txt        
  inflating: nucc/data007.txt        
  inflating: nucc/data008.txt        
  inflating: nucc/data009.txt        
  inflating: nucc/data010.txt        
  inflating: nucc/data011.txt        
  inflating: nucc/data012.txt        
  inflating: nucc/data013.txt        
  inflating: nucc/data014.txt        
  inflating: nucc/data015.txt        
  inflating: nucc/data016.txt        
  inflating: nucc/data017.txt        
  inflating: nucc/data018.txt        
  inflating: nucc/data019.txt        
  inflating: nucc/data020.txt        
  inflating: nucc/data021.txt        
  inflating: nucc/data022.txt        
  inflating: nucc/data023.txt        
  inflating: nucc/data024.txt        
  inflating: nucc/data025.txt        
  inflating:

# データセットの作成

## 作り方

- @や%で始める文は会話ではないのでskipする
- 発言者名のところは切り取る
- （）のなかにフーンとか補足見たいのがあるのでのける
- 会話文１つを出力と入力につかうので実質データセットは二倍
- ここの処理が雑かも、なのでうまくいかなかったら修正する

In [14]:
def make_data(file_name, data):
    input_line = None
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        if line[0] == '＠' or line[0] == '％': continue
        fix_s = ''
        skip = False
        for s in line:
            if s == '（':
                skip = True
            if skip:continue
            fix_s += s
            if s == '）':
                skip = False
        line = fix_s
        if len(line) >= 1:
            if line[0] == 'F' or line[0] == 'M':
                if input_line is None:
                    input_line = line[5:]
                else:
                    input_seq = input_line
                    output_seq = line[5:]
                    input_line = line[5:]
                    data.append((input_seq, output_seq))
            else:
                if input_line is None:
                    input_line = line
                else:
                    input_seq = input_line
                    output_seq = line
                    input_line = line
                    data.append((input_seq, output_seq))

            
    print(data)


In [15]:
import glob
file_name_list = glob.glob("nucc/*.txt")

data = []
for i in range(len(file_name_list)):
    make_data(file_name_list[i], data)
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



形態素解析を行う

In [17]:
# MeCabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

# mecab-ipadic-NEologdのインストール
!apt install git make curl xz-utils file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

# Ref: https://qiita.com/Fulltea/items/90f6ebe6dcceaf64eaef
# Ref: https://qiita.com/SUZUKI_Masaya/items/685000d569452585210c

!ln -s /etc/mecabrc /usr/local/etc/mecabrc
# Ref: https://qiita.com/Naritoshi/items/8f55d7d5cce9ce414395

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 277 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic-utf8 all 7.0-20130310-4 [16.2 MB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd6

In [21]:
import MeCab
wakati = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
    words = wakati.parse(sentence).split()
    return words

data_wakati = []
for d in data:
    data_wakati.append((make_wakati(d[0]), make_wakati(d[1])))

In [29]:
def create_dict(dataset):
    # 単語の出現回数を調べる
    word_cnt = {}
    for d in dataset:
        for i in range(2):
            for word in d[i]:
                if word in word_cnt:
                    word_cnt[word] += 1
                else:
                    word_cnt[word] = 1
    # 単語が3回以下しか出現しない単語にはindexをふらない
    word_dict = {}
    word_dict['<pad>'] = 0
    for k, v in word_cnt.items():
        if v > 3:
            word_dict[k] = len(word_dict)
    word_dict["<unk>"] = len(word_dict)
    return word_dict

In [30]:
word_dict = create_dict(data_wakati)
print(word_dict)

{'<pad>': 0, '俳句': 1, 'は': 2, 'いつ': 3, '始め': 4, 'た': 5, 'の': 6, '。': 7, '隣': 8, '、': 9, '老人': 10, '会': 11, 'で': 12, 'ね': 13, '隣組': 14, 'あの': 15, '人': 16, 'や': 17, '何': 18, 'か': 19, 'に': 20, '行っ': 21, 'てる': 22, 'とき': 23, '五': 24, '七': 25, 'だ': 26, 'から': 27, 'ふーん': 28, 'え': 29, '一番': 30, '最初': 31, '作っ': 32, 'って': 33, 'どん': 34, '覚え': 35, '？': 36, 'ねえ': 37, '持っ': 38, 'て': 39, 'いっ': 40, '＜': 41, '笑い': 42, '＞': 43, 'なー': 44, 'ん': 45, 'だっ': 46, 'たろ': 47, 'う': 48, 'じゃ': 49, 'ノート': 50, '書い': 51, 'いや': 52, 'その': 53, 'ずーっと': 54, 'うん': 55, 'いい': 56, 'よ': 57, 'これ': 58, 'ここ': 59, 'ある': 60, '今年': 61, 'お正月': 62, '初': 63, '釜': 64, '客': 65, 'はかま': 66, '立ち': 67, 'だから': 68, 'が': 69, 'し': 70, 'ほか': 71, 'はね': 72, '皆': 73, 'もう': 74, '年寄り': 75, 'りっぱ': 76, 'ねー': 77, 'はか': 78, 'まで': 79, '必ず': 80, 'それで': 81, 'いつも': 82, 'ほら': 83, 'Ｄ': 84, 'さん': 85, 'あたし': 86, 'と': 87, '親しく': 88, 'いる': 89, 'うち': 90, 'お茶': 91, 'たて': 92, 'を': 93, '先生': 94, 'お': 95, 'ばあちゃん': 96, '着物': 97, '着': 98, 'ああ': 99, 'あ': 100, '行く': 101, 'すばらし